In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#Importa código dos municípios do IBGE
cod_mun_ibge = pd.read_excel('C:/Users/franc/Desktop/Faculdade/Dados/Tabela_codigos_IBGE/RELATORIO_DTB_BRASIL_MUNICIPIO.xls')
ibge = cod_mun_ibge[['Nome_UF', 'Nome_Município', 'Código Município Completo']]
ibge.head()

In [ ]:
#Importa o dicionário das variáveis da base de conexão de internet
dicionario_ibge = pd.read_csv("C:/Users/franc/Desktop/Faculdade/Dados/Trabalho ICD/dic_ibge.csv")
dicionario_ibge.head()       

In [ ]:
#Importa a base de conexão de internet por município
internet_connection = pd.read_csv("C:/Users/franc/Desktop/Faculdade/Dados/Trabalho ICD/simet_veloc-2020-12-17-13-00.csv", 
                                  sep = ';')
internet_connection.head()

#DICIONÁRIO SIMET_DATA
#- cod_setor: setor censitário
#- lat, lon: coordenadas geográficas
#- asn: identificador da operadora
#- asn_name: nome da operadora
#- type: tipo de rede testada (mobile para rede móvel/broadband para banda larga)
#- averageTcp: velocidade média, em MB/s
#- numDisps: número de dispositivos que compõem a amostra
#- raio: raio considerado na região amostrada, em metros.
#- tcp_range_<velocidade>: número de dispositivos por faixa de velocidade de download, em MB/s.
#- lat_range_<ping>: número de dispositivos por faixa de latência, em milissegundos.
#- packLoss_range_<faixa_de_perda>: número de dispositivos por faixa de percentual de perda de pacotes.
#- jitDown_range_<faixa_de_jitter>: número de dispositivos por faixa de jitter de download, em milissegundos.
#- jitUp_range_<faixa_de_jitter>: número de dispositivos por faixa de jitter de upload, em milissegundos.

In [ ]:
#Juntando as bases
#Primeiro juntaremos a base de conexão de internet com o dicionário do IBGE

internet_connection_2 = internet_connection.merge(dicionario_ibge[['cod_sc', 'cod_mun']], how = "left", left_on = 'cod_setor', right_on = 'cod_sc')
internet_connection_2.head(7)

In [ ]:
internet_connection_2['vel_int'] = internet_connection_2['averageTcp'] * internet_connection_2['numDisps']
internet_connection_2 = internet_connection_2.drop(['averageTcp', 'numDisps'], axis = 1)

internet_connection_2 = internet_connection_2.groupby('cod_mun').agg({
    'vel_int': 'sum',
    'asn_name': 'nunique'
}).reset_index()

internet_connection_2.columns.values[2] = 'Num_operadoras'

internet_connection_2.head(7)

In [ ]:
#Importa dados do IDEB por município (anos iniciais)

ideb_anos_iniciais = pd.read_excel("C:/Users/franc/Desktop/Faculdade/Dados/INEP/Resultados IDEB/divulgacao_anos_iniciais_municipios_2019.xlsx",
                                  sep = ";", header = [6, 7, 8])


for i in range(len(ideb_anos_iniciais.columns.values)):
    col = list(ideb_anos_iniciais.columns.values[i])
    #Transforma títulos em string
    for j in range(3):
        if(not(isinstance(col[j], str))):
            col[j] = str(col[j])
    #Retira títulos vazios
    for j in range(3):
        if 'Unnamed' in col[j]:
            col[j] = ""
    col = tuple(col)
    ideb_anos_iniciais.columns.values[i] = col

#Junta os títulos hierárquicos em um só
ideb_anos_iniciais.columns = [' '.join(col).strip() for col in ideb_anos_iniciais.columns.values]

ideb_anos_iniciais.head(13)

In [ ]:
ideb_anos_iniciais.columns
rede_publica = ideb_anos_iniciais['Rede'] == 'Pública'
variaveis = ['Código do Município', 'Taxa de Aprovação - 2019  5º',  'Nota SAEB - 2019 Matemática', 'Nota SAEB - 2019 Língua Portuguesa', 'IDEB\n2019\n(N x P)']
ideb_anos_iniciais = ideb_anos_iniciais[rede_publica][variaveis]
novas_colunas = ['Cod_mun', 'Tax_aprov_5', 'SAEB_Mat_anos_iniciais', 'SAEB_port_anos_iniciais', 'IDEB_anos_iniciais']
for i in range(len(novas_colunas)):
    ideb_anos_iniciais.columns.values[i] = novas_colunas[i]

ideb_anos_iniciais.head()

In [ ]:
#Importa dados do IDEB (anos finais)
ideb_anos_finais = pd.read_excel("C:/Users/franc/Desktop/Faculdade/Dados/INEP/Resultados IDEB/divulgacao_anos_finais_municipios_2019.xlsx",
                                  sep = ";", header = [6, 7, 8])

for i in range(len(ideb_anos_finais.columns.values)):
    col = list(ideb_anos_finais.columns.values[i])
    #Transforma títulos em string
    for j in range(3):
        if(not(isinstance(col[j], str))):
            col[j] = str(col[j])
    #Retira títulos vazios
    for j in range(3):
        if 'Unnamed' in col[j]:
            col[j] = ""
    col = tuple(col)
    ideb_anos_finais.columns.values[i] = col

#Junta os títulos hierárquicos em um só
ideb_anos_finais.columns = [' '.join(col).strip() for col in ideb_anos_finais.columns.values]
#Tira a 1 linha (aba de pesquisa no excel)
ideb_anos_finais = ideb_anos_finais.drop(0, axis = 0)

ideb_anos_finais.head(13)


In [ ]:
ideb_anos_finais.columns
rede_publica = ideb_anos_finais['Rede'] == 'Pública'
variaveis = ['Código do Município', 'Taxa de Aprovação - 2019  9º', 'Nota SAEB - 2019 Matemática', 'Nota SAEB - 2019 Língua Portuguesa', 'IDEB\n2019\n(N x P)']
ideb_anos_finais = ideb_anos_finais[rede_publica][variaveis]

novas_colunas = ['Cod_mun', 'Aprov_9', 'SAEB_mat_anos_finais', 'SAEB_port_anos_finais', 'IDEB_anos_finais']
for i in range(len(novas_colunas)):
    ideb_anos_finais.columns.values[i] = novas_colunas[i]
ideb_anos_finais.head()


In [ ]:
#Importa dados do IDEB (ensino médio)
ideb_ensino_medio = pd.read_excel("C:/Users/franc/Desktop/Faculdade/Dados/INEP/Resultados IDEB/divulgacao_ensino_medio_municipios_2019.xlsx",
                                  sep = ";", header = [6, 7, 8])

for i in range(len(ideb_ensino_medio.columns.values)):
    col = list(ideb_ensino_medio.columns.values[i])
    #Transforma títulos em string
    for j in range(3):
        if(not(isinstance(col[j], str))):
            col[j] = str(col[j])
    #Retira títulos vazios
    for j in range(3):
        if 'Unnamed' in col[j]:
            col[j] = ""
    col = tuple(col)
    ideb_ensino_medio.columns.values[i] = col

#Junta os títulos hierárquicos em um só
ideb_ensino_medio.columns = [' '.join(col).strip() for col in ideb_ensino_medio.columns.values]
#Tira a 1 linha (aba de pesquisa no excel)
ideb_ensino_medio = ideb_ensino_medio.drop(0, axis = 0)

ideb_ensino_medio.head(13)


In [ ]:
ideb_ensino_medio.columns
rede_publica = ideb_ensino_medio['Rede'] == 'Pública'
variaveis = ['Código do Município', 'Taxa de Aprovação - 2019  Total', 'Nota SAEB - 2019 Matemática', 'Nota SAEB - 2019 Língua Portuguesa', 'IDEB\n2019\n(N x P)']
ideb_ensino_medio = ideb_ensino_medio[rede_publica][variaveis]

novas_colunas = ['Cod_mun', 'Aprov_e_m', 'SAEB_mat_e_m', 'SAEB_port_e_m', 'IDEB_e_m']
for i in range(len(novas_colunas)):
    ideb_ensino_medio.columns.values[i] = novas_colunas[i]
    
ideb_ensino_medio.head()


In [ ]:
#Importa variáveis de controle para os municípios
variaveis_controle = pd.read_csv("C:/Users/franc/Desktop/Faculdade/Dados/Trabalho ICD/variaveis_de_controle.csv",
                                sep = ',')
variaveis_controle.head()

In [ ]:
#Vamos selecionar variáveis de anos diferentes:
#Censo 2010 (% de pobres, taxa de analfabetismo)
#DataSUS 2017 (Taxa de mortalidade infantil, % de meninas que tiveram filhos)
#RAIS 2016 (PIB per capita)
variaveis = ['Territorialidades', 'Esperança de vida ao nascer 2010', 'Taxa de analfabetismo - 15 anos ou mais de idade 2010', '% de pobres 2010', 
             'Produto Interno Bruto per capita 2016', 'Taxa de mortalidade infantil 2017', '% de meninas de 10 a 14 anos de idade que tiveram filhos 2017']
variaveis_controle = variaveis_controle[variaveis].drop(0, axis = 0)
colunas_novas = ['Município', 'Exp_vida', 'analfabetismo', 'pob', 'PIBpercap', 'mort_inf', 'meninas_filhos']
for i in range(len(colunas_novas)):
    variaveis_controle.columns.values[i] = colunas_novas[i]

variaveis_controle.head()

In [ ]:
cod_mun_ibge[cod_mun_ibge['Nome_UF'] == 'Goiás'].head()
variaveis_controle.shape

In [ ]:
#Tirando o nome dos estados das strings de municípios da base de variáveis de controle

for i in range(variaveis_controle.shape[0]):
    variaveis_controle.iloc[i,0] = variaveis_controle.iloc[i,0][:(len(variaveis_controle.iloc[i,0]) - 5)]

#Dá match com o código do município no IBGE:

variaveis_controle = variaveis_controle.merge(cod_mun_ibge, how = 'right', left_on = 'Município', right_on = 'Nome_Município')
variaveis_controle = variaveis_controle.drop(['Município', 'Nome_UF', 'Nome_Município'], axis = 1)
variaveis_controle.head()

In [ ]:
#Parte 2: Juntando as bases


In [ ]:
#Juntando com as bases do IDEB e do IBGE
df = internet_connection_2.merge(ideb_anos_iniciais, how = 'left', left_on = 'cod_mun', right_on = 'Cod_mun').merge(ideb_anos_finais, 
how = 'left', left_on = 'cod_mun', right_on = 'Cod_mun').merge(ideb_ensino_medio, how = 'left', left_on = 'cod_mun', 
right_on = 'Cod_mun').merge(cod_mun_ibge, how = 'left', left_on = 'cod_mun', right_on = 'Código Município Completo')

df = df.drop(['Cod_mun_x', 'Cod_mun_y', 'Cod_mun', 'Código Município Completo'], axis = 1)

df.head()

In [ ]:
#Juntando com a base de variáveis de controle
df = df.merge(variaveis_controle, how = 'left', left_on = 'cod_mun', right_on = 'Código Município Completo')


In [ ]:
#Reorganizando as colunas
ordem_colunas = ['Nome_UF', 'cod_mun', 'Nome_Município', 'vel_int', 'Num_operadoras', 'Tax_aprov_5','SAEB_Mat_anos_iniciais', 'SAEB_port_anos_iniciais','IDEB_anos_iniciais', 'Aprov_9', 'SAEB_mat_anos_finais','SAEB_port_anos_finais', 'IDEB_anos_finais', 'Aprov_e_m','SAEB_mat_e_m', 'SAEB_port_e_m', 'IDEB_e_m']
df = df[ordem_colunas]
df.head()